# AICE Professioanl 대비 - 텍스트 처리(NLP) 실습
- 네이버 영화 리뷰 데이터 기반 영화리뷰에 대한 긍정(1), 부정(0) 라벨 감성분석 실습

## NLP 순서
- 텍스트 데이터 전처리
- 한글 형태소 분석(Okt, Mecab 등)
- Tokenizer(Word Dictionary 생성)
- Texts to Sequences
- Embedding(단어 밀집벡터 표현)
- LSTM 모델링


## 모델링 코드 순서

**1. 데이터 로드 및 공통 전처리**
- train_data, test_data Load
- Preprocess -> 결측치 제거 및 텍스트 정제(한글 외 문자 제거)

**2. 머신러닝**
- 2.1 머신러닝 모델링
- 2.2 머신러닝 모델 평가 및 예측

**3. 딥러닝 모델링**
- 3.1 전처리: 1)형태소 분석(Okt), 2)단어 인덱스화(Tokenizer), 3)Padding 처리
- 3.2 딥러닝 모델링
- 3.3 딥러닝 모델 평가 및 예측



In [1]:
!pip install konlpy # konlpy 설치 (최초 실행 시 필요)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 29.0 MB/s eta 0:00:00


# 1. 데이터 로드 및 공통 전처리
## 데이터 로드
먼저, NSMC 데이터셋을 Colab 환경에 다운로드하고 로드합니다.

In [2]:
# 필요한 라이브러리 임포트
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from tqdm import tqdm
import tensorflow as tf

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

('ratings_test.txt', <http.client.HTTPMessage at 0x7d4e761354b0>)

In [4]:
# 데이터 로드 (NSMC 데이터셋 다운로드 필요)
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')


## 데이터 확인
데이터의 첫 몇 행과 결측값 유무를 확인합니다.

In [5]:
# 데이터 확인
print(train_data.head())

         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1


In [6]:
train_data['document'][:50]

,document
0,아 더빙.. 진짜 짜증나네요 목소리
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2,너무재밓었다그래서보는것을추천한다
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
5,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.
6,원작의 긴장감을 제대로 살려내지못했다.
7,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...
8,액션이 없는데도 재미 있는 몇안되는 영화
9,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?


In [7]:
print(train_data.isnull().sum())

id          0
document    5
label       0
dtype: int64


## 결측값 제거 및 텍스트 정제
결측값을 제거하고 한글 외의 문자들을 삭제하여 텍스트를 정제합니다.

In [8]:
# 결측값 제거
train_data = train_data."""입력하세요"""(how='any')
test_data = test_data.dropna(how='any')

SyntaxError: invalid syntax (<ipython-input-8-d2d8b2a99a45>, line 2)

In [ ]:
print(train_data."""입력하세요"""().sum())
print(test_data.isnull().sum())

In [ ]:
# 한글 외 문자 제거
train_data['document'] = train_data['document'].str."""입력하세요"""("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)
test_data['document'] = test_data['document'].str.replace("""입력하세요""","", regex=True)

In [ ]:
train_data['document'][:50]



---

---



# 2. 머신러닝
## 2.1 머신러닝 모델링

머신러닝 모델을 사용할 때:
TF-IDF와 같은 벡터화 기법을 사용하여 고정 길이의 입력 데이터를 생성합니다.

단어의 순서나 문맥 정보가 크게 중요하지 않을 때 효과적입니다.

### TF-IDF 벡터화
텍스트 데이터를 TF-IDF 벡터로 변환합니다.

In [ ]:
# TF-IDF 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터라이저 생성
vectorizer = """입력하세요"""()
X_train_tfidf = vectorizer."""입력하세요"""(train_data['document'])
X_test_tfidf = vectorizer."""입력하세요"""(test_data['document'])

# 레이블 추출
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])


## 머신러닝 모델 학습
로지스틱 회귀 모델을 사용하여 학습합니다.

In [ ]:
# 로지스틱 회귀 모델 학습
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression()
lr_model.fit(X_train_tfidf, y_train)


## 2.2 머신러닝 모델 평가 및 예측
모델 평가
테스트 데이터를 사용하여 머신러닝 모델의 성능을 평가합니다.

In [ ]:
# 모델 평가
from sklearn.metrics import accuracy_score

y_pred_ml = lr_model."""입력하세요"""(X_test_tfidf)
accuracy_ml = """입력하세요"""(y_test, y_pred_ml)
print('머신러닝 모델 테스트 정확도: {:.4f}'.format(accuracy_ml))


In [ ]:
# 예측 함수 정의
def ml_sentiment_predict(new_sentence):
    new_sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", new_sentence)
    vector = vectorizer.transform([new_sentence])
    result = lr_model.predict(vector)
    if result == 1:
        print("긍정 리뷰입니다.")
    else:
        print("부정 리뷰입니다.")


In [ ]:
ml_sentiment_predict('이 영화 정말 재미있어요!')
ml_sentiment_predict('별로였어요. 추천하지 않습니다.')



---



---



# 3. 딥러닝

## 3.1 딥러닝 전처리



## 형태소 분석 및 토큰화
Okt 형태소 분석기를 사용하여 문장을 단어 단위로 분할하고, 불용어를 제거합니다.

In [ ]:
# 형태소 분석기 이용한 토큰화
from konlpy.tag import Okt
okt = Okt()

# 형태소 분석 함수 정의
def okt_morphs(text):
    return okt.morphs(text, stem=True)  # 어간 추출을 포함한 형태소 분석

# 토큰화 적용
train_data['tokenized'] = train_data['document'].apply("""입력하세요""")
test_data['tokenized'] = test_data['document'].apply("""입력하세요""")


In [ ]:
train_data

## 단어 인덱스화
딥러닝 모델을 사용할 때:
Tokenizer와 임베딩 레이어를 사용하여 단어의 순서와 문맥을 고려합니다.

복잡한 패턴과 의미를 학습하고자 할 때 적합합니다.

각 단어를 고유한 정수로 인코딩합니다.

In [ ]:
# 전체 단어 사전 구축
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = """입력하세요"""()
tokenizer.fit_on_texts(train_data['tokenized'])

# 텍스트를 시퀀스로 변환
X_train = tokenizer."""입력하세요"""(train_data['tokenized'])
X_test = tokenizer.texts_to_sequences(test_data['tokenized'])

# 레이블 추출
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])


## 패딩 처리
딥러닝 모델에서는 입력 시퀀스의 길이를 동일하게 맞추기 위해 패딩을 적용합니다.

In [ ]:
# 패딩 처리
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_len = max(len(line) for line in X_train)  # 최대 문장 길이 지정
X_train_padded = """입력하세요"""(X_train, maxlen=max_len)
X_test_padded = pad_sequences(X_test, maxlen=max_len)


## 3.2 딥러닝 모델링
딥러닝 모델에서는 임베딩 층과 순환 신경망을 사용합니다.

**모델 구성**

임베딩 층과 LSTM 층을 활용하여 모델을 구성합니다.

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPool2D
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, SimpleRNN, GRU, Dropout, SpatialDropout1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# 모델 구성
"""
dl_model = Sequential()
dl_model.add(Embedding(len(tokenizer.word_index)+1, 100))  # 단어 사전 크기, 임베딩 벡터 차원
dl_model.add(LSTM(128))  # LSTM 레이어
dl_model.add(Dense(1, activation='sigmoid'))  # 출력층
"""
# 6. 텍스트 토큰화 및 시퀀스 변환
max_words = 35000  # 사전에 사용할 최대 단어 수
max_len = max(len(line) for line in X_train)       # 패딩 시퀀스의 최대 길이

embedding_dim = 128  # 임베딩 차원
lstm_units = 128     # LSTM 유닛 수

dl_model = Sequential()
dl_model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))  # 단어 사전 크기, 임베딩 벡터 차원
# LSTM 레이어
dl_model.add(SpatialDropout1D(0.3))  # 드롭아웃 추가
dl_model.add(LSTM(lstm_units, dropout=0.3, recurrent_dropout=0.3))
dl_model.add(Dense(1, activation='sigmoid'))  # 출력층

# 모델 컴파일
dl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
dl_model.summary()

## 모델 학습
훈련 데이터를 사용하여 모델을 학습시킵니다.

In [ ]:
# 조기종료 콜백함수 정의(EarlyStopping)
es = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# 체크포인트 저장(ModelCheckpoint)
checkpoint_path = 'tmp_checkpoint.keras'
cp = ModelCheckpoint(filepath = checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True)

In [ ]:

# history = dl_model.fit(X_train_padded, y_train, epochs=5, batch_size=64, validation_split=0.2)
# 모델 학습(fit)
history = dl_model.fit(X_train_padded, y_train, epochs=5, batch_size=512, validation_split=0.2, verbose =1, callbacks=[es, cp])


In [ ]:
##모델 로드 웨이트
dl_model.load_weights(checkpoint_path)
##모델 저장
dl_model.save("NLP_mission_1.h5")

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], )
plt.show()

## 3.3 딥러닝 모델 평가 및 예측
모델 평가
테스트 데이터를 사용하여 딥러닝 모델의 성능을 평가합니다.

In [ ]:
# 모델 평가
loss, accuracy = dl_model.evaluate(X_test_padded, y_test)
print('딥러닝 모델 테스트 정확도: {:.4f}'.format(accuracy))


## 예측 함수 정의
새로운 문장에 대한 감성 예측을 수행하는 함수를 정의합니다.

In [ ]:
# 예측 함수 정의
def dl_sentiment_predict(new_sentence):
    new_sentence = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True)
    encoded = tokenizer."""입력하세요"""([new_sentence])
    pad_new = """입력하세요"""(encoded, maxlen=max_len)
    score = float(dl_model.predict(pad_new))
    if score > 0.5:
        print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))


## 예시 문장 예측
정의한 함수를 사용하여 새로운 문장의 감성을 예측합니다.

In [ ]:
# 예시 문장 예측
dl_sentiment_predict('이 영화 정말 재미있어요!')
dl_sentiment_predict('별로였어요. 추천하지 않습니다.')
